基线组流程
```
data   :c37      -> splits   -> target_test -> infer    -> eval

process:Original -> prepare  -> split       -> infer    -> cell-eval

.X     :raw      -> prepared -> original    -> inferred -> evaluated
```

数据变化
- 对齐组：使用对齐后的数据进行推理
- 基线组：使用未对齐的数据进行推理

## 1.数据准备

## 2.数据拆分

## 3.基线组推理

In [ ]:
# 使用未对齐的数据进行推理，作为对照基线
export CUDA_VISIBLE_DEVICES=4
state tx infer \
  --model-dir /data3/fanpeishan/state/for_state/models/ST-Tahoe \
  --checkpoint /data3/fanpeishan/state/for_state/models/ST-Tahoe/final_from_preprint.ckpt \
  --adata /data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad \
  --output /data3/fanpeishan/state/for_state/run_results/run21/target_test_infer.h5ad \
  --pert-col drugname_drugconc \
  --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
  --quiet \
  --batch-col plate 

In [ ]:
=== Inference complete ===
Input cells:         321297
Controls simulated:  7511
Treated simulated:   313786
Wrote predictions to adata.X
Saved:               /data3/fanpeishan/state/for_state/run_results/run21/target_test_infer.h5ad

## 4.基线组评估

In [ ]:
# 使用 cell-eval 评估基线组的推理结果
export CUDA_VISIBLE_DEVICES=4
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/run_results/run21/target_test_infer.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad \
    -o /data3/fanpeishan/state/for_state/run_results/run21/run_results/ \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile minimal \
    --batch-size 1024 \
    --num-threads 12 \
    --skip-metrics de_nsig_counts_real,de_nsig_counts_pred

In [1]:
import pandas as pd
results = pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run21/run_results/agg_results.csv')
results[results.statistic == 'mean']

,statistic,overlap_at_N,precision_at_N,de_nsig_counts_real,de_nsig_counts_pred,pearson_delta,mse,mae,discrimination_score_l1
2,mean,0.132985,0.175033,192.318872,522.067245,0.391485,0.112473,0.077353,0.619176
